In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import logging 

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import json
import polars as pl
from polars import col, lit
import altair as alt
import simrs
from history import History, tabular
import input_builder as b
import utils 

In [10]:
# Agents
agent_ids = ["a" +str(i) for i in range(1, 5)]

# Goods
wheat = "Wheat"
goods = [wheat]

# Ports
genoa = "Genoa"
rome = "Rome"
milan = "Milan"
venice = "Venice"
marsailles = "Marsailles"
port_ids = [genoa, rome, milan, venice, marsailles]

# genoa -> milan -> marsailles
#   v         v
# rome -> venice
edges = [(genoa, milan), (milan, marsailles), (rome, venice), (rome, genoa), (venice, milan)]
_market = lambda net: b._market(wheat, b._market_info(net=net)) 

# net balanced
ports = [
  b._port(genoa, _market(10)), 
  b._port(milan, _market(5)), 
  b._port(rome, _market(0)), 
  b._port(venice, _market(-5)),
  b._port(marsailles, _market(-10)), 
]

_agent = lambda id, pos: b._agent(id, pos, 1000, "Greedy")
agents = [_agent(id, port_id) for (id, port_id) in zip(agent_ids, port_ids)]

x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=10))
history = simrs.run(x)
(actions, agents, markets) = tabular(history)

In [11]:
history.history["states"][-1]["agents"]

{'a3': {'id': 'a3',
  'pos': 'Genoa',
  'cargo': None,
  'coins': 988.6000000000001,
  'behavior': 'Greedy'},
 'a4': {'id': 'a4',
  'pos': 'Milan',
  'cargo': None,
  'coins': 996.6000000000001,
  'behavior': 'Greedy'},
 'a2': {'id': 'a2',
  'pos': 'Genoa',
  'cargo': None,
  'coins': 984.1999999999998,
  'behavior': 'Greedy'},
 'a1': {'id': 'a1',
  'pos': 'Milan',
  'cargo': None,
  'coins': 996.1999999999998,
  'behavior': 'Greedy'}}

In [12]:
markets.select(col("pricer").struct.field("base_price")).head()
markets.select(col("pricer")).schema

{'pricer': Struct([Field('base_price', Float64), Field('base_supply', Float64), Field('price_per_supply', Float64)])}

In [13]:
def plot_agents(agents: pl.DataFrame):
    base = alt.Chart(agents).encode(
        x='tick:Q',
        y=alt.Y('coins:Q').scale(zero=False),
        color='id:O',
    )
    lines = base.transform_loess('tick', 'coins', bandwidth=.5, groupby=['id']).mark_line(size=4)
    return (base.mark_point() + lines).interactive() 
  
plot_agents(agents)

alt.LayerChart(...)

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)